In [66]:
import redis
import json
import datetime
import math

class Cache():
    
    def __init__(self):
        self.r = redis.StrictRedis(host='localhost', port = 6379, db = 0)
        self.p = redis.StrictRedis(host='localhost', port = 6379, db = 1)

    def data_modifier(self, data):
        res = {'res': data}
        json_res = json.dumps(res,ensure_ascii=False).encode('utf8')
        return json_res

    def convert_to_num(self, s):
        if s[-1] == 'K':
            return (float(s[0:s.find('.')]) + float(s[s.find('.'):s.find('K')]))/1000
        return float(s[0:s.find('.')]) + float(s[s.find('.'):s.find('M')])

    def clear_20_percent(self, cur_mem):

        # First sorting searches
        keys, idle_times = [], []
        for key in self.r.scan_iter("*"):
            idle = self.r.object("idletime", key)
            keys.append(key)
            idle_times.append(idle)

        keys = [k for i,k in sorted(zip(idle_times, keys))]
        target = math.ceil(len(keys)*0.2)

        # deleting oldest 20% of keys
        for i in range(target):
            self.r.delete(keys.pop())

        # if current cache memory still more than 10 mb [only a safety check]
        cur_mem = self.convert_to_num(self.r.info()['used_memory_human'])
        while(cur_mem > 10):
            self.r.delete(keys.pop())
            cur_mem = self.convert_to_num(self.r.info()['used_memory_human'])

    def push_to_dynamic_cache(self, key, data):
        data = self.data_modifier(data)
        cur_mem = self.convert_to_num(self.r.info()['used_memory_human'])
        if cur_mem < 10:
            self.r.set(key, data)
        else:
            self.clear_20_percent(cur_mem)
            self.r.set(key, data)

    def get_from_dynamic_cache(self, key):
        try:
            return json.loads(self.r.get(key).decode())['res']
        except:
            return -1

    def push_to_top10_cache(self, key, data):
        data = self.data_modifier(data)
        self.p.set(key, data)

    def get_from_top10_cache(self, key):
        try:
            return json.loads(self.p.get(key).decode())['res']
        except:
            return -1

In [67]:
o1=Cache()

In [76]:
x=[(62513246,
  '62513246',
  'J.K. Rowling',
  'jk_rowling',
  'Scotland',
  'Writer sometimes known as Robert Galbraith',
  'True',
  14608046,
  27353,
  (datetime.datetime(2009, 8, 3, 13, 23, 45)).isoformat(),
  721),
 (186712788,
  '186712788',
  'jose marie viceral',
  'vicegandako',
  'Earth',
  'Supermodel turned actress turned horse',
  'True',
  13527555,
  135,
  (datetime.datetime(2010, 9, 4, 5, 12, 31)).isoformat(),
  517),
   (405427035,
  '405427035',
  'Arvind Kejriwal',
  'ArvindKejriwal',
  'India',
  'सब इंसान बराबर हैं, चाहे वो किसी धर्म या जाति के हों। हमें ऐसा भारत बनाना है जहाँ सभी धर्म और जाति के लोगों में भाईचारा और मोहब्बत हो, न कि नफ़रत और बैर हो।',
  'True',
  18335920,
  618,
  (datetime.datetime(2011, 11, 5, 9, 15, 8)).isoformat(),
  221)]

In [79]:
o1.push_to_dynamic_cache('search rowling category author',x)

In [80]:
o1.get_from_dynamic_cache('search rowling category author')

[[62513246,
  '62513246',
  'J.K. Rowling',
  'jk_rowling',
  'Scotland',
  'Writer sometimes known as Robert Galbraith',
  'True',
  14608046,
  27353,
  '2009-08-03T13:23:45',
  721],
 [186712788,
  '186712788',
  'jose marie viceral',
  'vicegandako',
  'Earth',
  'Supermodel turned actress turned horse',
  'True',
  13527555,
  135,
  '2010-09-04T05:12:31',
  517],
 [405427035,
  '405427035',
  'Arvind Kejriwal',
  'ArvindKejriwal',
  'India',
  'सब इंसान बराबर हैं, चाहे वो किसी धर्म या जाति के हों। हमें ऐसा भारत बनाना है जहाँ सभी धर्म और जाति के लोगों में भाईचारा और मोहब्बत हो, न कि नफ़रत और बैर हो।',
  'True',
  18335920,
  618,
  '2011-11-05T09:15:08',
  221]]

In [71]:
o1.push_to_top10_cache('top10users', x)

In [72]:
o1.get_from_top10_cache('top10authors')

-1